In [1]:
import numpy as np
import pandas as pd
from numpy import random

import re

import random

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오는 부분

area = 'Gangnam' # 데이터 불러오는 지역
data = pd.read_csv(f'../../crawings/user_csv/{area}/user_concat.csv')
data['userid'] = data['link'].apply(lambda x: str(re.findall(r'my/[a-z0-9]+/', x))[5:-3])
data = data[['userid', 'rest']]
data = data.drop_duplicates()

In [3]:
# train/ test 분할하는 부분

data['rand'] = data['rest'].apply(lambda x : random.random())
_user = data['userid'].value_counts().reset_index()
_user.columns = ['userid', 'cnt']
data = pd.merge(data, _user, how = 'left', on = 'userid')
data = data[data['cnt'] > 5].reset_index(drop = True)
data = data[~(data['userid'] == '')].reset_index(drop = True)
data = data.sort_values(['userid', 'rand']).reset_index(drop = True)
data['tem'] = 1
data['seq'] = data.groupby('userid')['tem'].apply(lambda x : x.cumsum())

train = data[data['tem'] + 1 < data['seq']]
test = data[data['tem'] + 1 >= data['seq']]

In [6]:
train[['userid', 'rest']].to_csv('G_train.csv', index = False)
test[['userid', 'rest']].to_csv('G_test.csv', index = False)